
# <p style="text-align: center;">EE 461P: Data Science Principles</p>
# <p style="text-align: center;">Assignment 5</p>
## <p style="text-align: center;">Total points: 40</p>
## <p style="text-align: center;">Due: Thursday, November 30th, submitted via Canvas by 11:59 pm</p>

Your homework should be written in a **Jupyter notebook**. You may work in groups of two if you wish. Only one student per team needs to submit the assignment on Canvas.  But be sure to include name and UTID for both students.  Homework groups will be created and managed through Canvas, so please do not arbitrarily change your homework group.  

Also, please make sure your code runs and the graphics (and anything else) are displayed in your notebook before submitting. (%matplotlib inline)

# Question 1: Ensembles (1+12+2 = 15pts)
In this question, we will compare performance of different ensemble methods: [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html), [Gradient Boosting Decision Tree](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html) (GBDT), and [XGBoost](http://xgboost.readthedocs.io/en/latest/).  Note that you have to install xgboost package in addition to scikit-learn.  You can see installation guides [here](http://xgboost.readthedocs.io/en/latest/build.html).




Two  datasets are provided for this problem. For **each of the datasets ((X1.csv, y1.csv), (X2.csv, y2.csv))**, do the following:

1. Load the data and partition it into features (X) and the target label (y) for classification task. Then, use [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to split data into training and testing: test_size=0.33, random_state=42.

2. Build a classifier using [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html), [Gradient Boosting Decision Tree](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html), and [XGBoost](http://xgboost.readthedocs.io/en/latest/), respectively, and answer the following for each classifier.

 - Mention any design choices (with reasoning/justification) that you made, e.g. the hyperparameters considered for each classifier.
 - Report the mean error rate (fraction of incorrect labels) and the confusion matrix on test data. <br>
 - Report the feature importance and time of execution (training and predicting times).

3. Compare the three classifiers for the two different datasets ((X1.csv, y1.csv), (X2.csv, y2.csv)) in terms of the misclassification rate.  What are the characteristics of the dataset and the classifiers that resulted in somewhat different comparative results?

In [1]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from xgboost.sklearn import XGBClassifier
from time import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


from sklearn.metrics import mean_squared_error, confusion_matrix, zero_one_loss
%matplotlib inline

/home/zhangtong/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Dataset 1

In [2]:
import pandas as pd
from sklearn.metrics import accuracy_score
#load and split the data
X = pd.read_csv("X1.csv").values
y = pd.read_csv("y1.csv").values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [3]:
y_test.shape[0]

3300

# Random Forest

In [4]:
clf_rfc = RandomForestClassifier()

In [5]:
parameters = {'n_estimators': [10, 20, 30, 40, 50]}

def mean_class_accuracy(estimator, X, y):
    cm = confusion_matrix(y, estimator.predict(X))
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    return np.mean(cm_normalized.diagonal())

clf = GridSearchCV(clf_rfc, parameters, cv=3, scoring=mean_class_accuracy)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

/home/zhangtong/.local/lib/python3.5/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/zhangtong/.local/lib/python3.5/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/zhangtong/.local/lib/python3.5/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/zhangtong/.local/lib/python3.5/site-packages/sklearn/model_selection/_validation.py:458: DataConvers

In [6]:
print("result of the random forest:\n")
print("Mean Error: {}".format(zero_one_loss(predictions, y_test)))
cm = confusion_matrix(y_test, predictions)
print(cm)
clf.best_params_

result of the random forest:

Mean Error: 0.12303030303030305
[[1448  252]
 [ 154 1446]]


{'n_estimators': 50}

#  GradientBoosting

In [7]:
#Gradient Boosting Decision Tree
clf_gbc = GradientBoostingClassifier(n_estimators=100)

In [8]:
parameters = {'learning_rate': [0.01, 0.1, 1]}

def mean_class_accuracy(estimator, X, y):
    cm = confusion_matrix(y, estimator.predict(X))
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    return np.mean(cm_normalized.diagonal())

clf = GridSearchCV(clf_gbc, parameters, cv=3, scoring=mean_class_accuracy)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

/home/zhangtong/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/zhangtong/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/zhangtong/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/zhangtong/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

In [9]:
print("result of the GradientBoosting:\n")
print("Mean Error: {}".format(zero_one_loss(predictions, y_test)))
cm = confusion_matrix(y_test, predictions)
print(cm)
clf.best_params_

result of the GradientBoosting:

Mean Error: 0.11909090909090914
[[1481  219]
 [ 174 1426]]


{'learning_rate': 0.1}

# XGBoost

In [28]:
#XGBoost(skip for now, because I didn't figure how to install XGboost, I will probally do this on my local computer)
clf_xgb = XGBClassifier(max_depth=30, learning_rate=0.1, n_estimators=50)

In [29]:
clf_xgb.fit(X_train, y_train)
predictions = clf_xgb.predict(X_test)

/home/zhangtong/.local/lib/python3.5/site-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/zhangtong/.local/lib/python3.5/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [30]:
print("result of the XGBoost:\n")
print("Mean Error: {}".format(zero_one_loss(predictions, y_test)))
cm = confusion_matrix(y_test, predictions)
print(cm)
clf.best_params_

result of the XGBoost:

Mean Error: 0.08060606060606057
[[1545  155]
 [ 111 1489]]


{'learning_rate': 0.1}

## Dataset 2

In [31]:
X = pd.read_csv("X2.csv").values
y = pd.read_csv("y2.csv").values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Random Forest

In [32]:
clf_rfc = RandomForestClassifier()

In [33]:
parameters = {'n_estimators': [10, 20, 30, 40, 50]}

def mean_class_accuracy(estimator, X, y):
    cm = confusion_matrix(y, estimator.predict(X))
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    return np.mean(cm_normalized.diagonal())

clf = GridSearchCV(clf_rfc, parameters, cv=3, scoring=mean_class_accuracy)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

/home/zhangtong/.local/lib/python3.5/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/zhangtong/.local/lib/python3.5/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/zhangtong/.local/lib/python3.5/site-packages/sklearn/model_selection/_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/zhangtong/.local/lib/python3.5/site-packages/sklearn/model_selection/_validation.py:458: DataConvers

In [34]:
print("result of the random forest:\n")
print("Mean Error: {}".format(zero_one_loss(predictions, y_test)))
cm = confusion_matrix(y_test, predictions)
print(cm)
clf.best_params_

result of the random forest:

Mean Error: 0.07878787878787874
[[782  72]
 [ 58 738]]


{'n_estimators': 40}

#  GradientBoosting

In [35]:
clf_gbc = GradientBoostingClassifier(n_estimators=100)

In [36]:
parameters = {'learning_rate': [0.01, 0.1, 1]}

def mean_class_accuracy(estimator, X, y):
    cm = confusion_matrix(y, estimator.predict(X))
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    return np.mean(cm_normalized.diagonal())

clf = GridSearchCV(clf_gbc, parameters, cv=3, scoring=mean_class_accuracy)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

/home/zhangtong/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/zhangtong/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/zhangtong/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/zhangtong/.local/lib/python3.5/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

In [37]:
print("result of the GradientBoosting:\n")
print("Mean Error: {}".format(zero_one_loss(predictions, y_test)))
cm = confusion_matrix(y_test, predictions)
print(cm)
clf.best_params_

result of the GradientBoosting:

Mean Error: 0.09333333333333338
[[761  93]
 [ 61 735]]


{'learning_rate': 1}

# XGBoost

In [48]:
clf_xgb = XGBClassifier(max_depth=50, learning_rate=1, n_estimators=100)

In [49]:
clf_xgb.fit(X_train, y_train)
predictions = clf_xgb.predict(X_test)

/home/zhangtong/.local/lib/python3.5/site-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/zhangtong/.local/lib/python3.5/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [50]:
print("result of the XGBoost:\n")
print("Mean Error: {}".format(zero_one_loss(predictions, y_test)))
cm = confusion_matrix(y_test, predictions)
print(cm)

result of the XGBoost:

Mean Error: 0.0757575757575758
[[778  76]
 [ 49 747]]


## Reasoning

For this problem, it required to use random forest, Gradient Boosting and XGBoost. I start with tuning the random forest using grid serach on the n_estimators, as a result  the best param is 50. I use the same approach on Gradient Boosting, but I am looking for the learning rate this time. the result is 0.1. Next, I use the params for XGBoost, which give the least mean error for dataset 1.

I also use this approach for dataset2

# Question 2: Visualization using Bokeh (10 pts)

In this problem, you'll build an interactive visualization. Bokeh is a Python interactive visualization library that targets modern web browsers for presentation. For more information on Bokeh, see http://bokeh.pydata.org/en/latest/. The problem statement is as follows:

Using the "nbasalariesfull.csv" data set from HMK4, your goal is to build a Bokeh visualization which allows the user to explore how salary (on a log scale) varies with points per game (PSG) and age. You will create a visualization that allows the user to toggle the X axis of a scatter plot between PSG and age, with the y-axis always being log Salary. Also add the hover tool so that if the user hovers over a datapoint in the plot a window pops up that shows the player name, team, position, salary, and the current x variable (PSG or age) depending on the current tab.  Color each point according to a player's position and provide a legend for the colors. Add the ability to Zoom in/out.  Add slight horizontal jitter to a player's age.

Hints: 
1. see: http://bokeh.pydata.org/en/latest/docs/user_guide/tools.html#basic-tooltips for hover and zoom tool examples.
2. See: http://bokeh.pydata.org/en/latest/docs/reference/plotting.html. Look for the scatter API.
3. See: http://bokeh.pydata.org/en/0.10.0/docs/user_guide/styling.html#labels. For labeling axes.
4. See: https://bokeh.pydata.org/en/latest/docs/user_guide/categorical.html  for how to use jitter transform
5. Use output_notebook() from Bokeh to output the plot to your notebook

In [3]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from bokeh.models.widgets import Panel, Tabs
from bokeh.models import CustomJS, ColumnDataSource, HoverTool, BoxZoomTool
from bokeh.transform import jitter
from bokeh.plotting import figure, output_notebook, show
from bokeh.plotting import ColumnDataSource
from bokeh.models import HoverTool, ZoomInTool, ZoomOutTool, PanTool, WheelZoomTool
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.transform import factor_cmap
from bokeh.plotting import figure
from bokeh.palettes import Set3_7, Category10_7, Spectral7
from bokeh.layouts import widgetbox
from ipywidgets import interact
from bokeh.io import push_notebook, show, output_notebook

output_notebook()
data = pd.read_csv("nbasalariesfull.csv")
data["logsalary"] = data.SALARY.apply(np.log)

Loading BokehJS ...

In [4]:
mycolor = Category10_7
def update(X_Variable, Age_Jitter):
    X = data.Age
    X_label = "Age"
    jitt = Age_Jitter
    Title = "NBA Salaries v. Age"
    
    if X_Variable == "PSG": 
        X = data.PSG
        X_label = "PSG"
        jitt = 0.0
        Title = "NBA Salaries v. PSG"
    source = ColumnDataSource(data=dict(
        x=X,
        y=data.SALARY.apply(np.log),
        Name = data.Player,
        Salary = data.SALARY,
        Team = data.Tm,
        Position = data.Pos,
    ))

    hover = HoverTool(tooltips=[
        ("Name", "@Name"),
        ("Team", "@Team"),
        ("Position", "@Position"),
        (X_label, "@x"),
        ("Salary", "$@Salary")
    ])

    wheelzoom = WheelZoomTool();
    plot = figure(title = Title, plot_width=600, plot_height=600, tools = [hover, ZoomInTool(), ZoomOutTool(), PanTool(),wheelzoom], active_scroll = wheelzoom)

    plot.scatter(x=jitter('x', width=jitt, range=plot.x_range), y='y', size=6, source=source, 
                 color=factor_cmap('Position', palette=mycolor, factors=['PG', 'SG', 'PF', 'SF', 'C', 'PF-C', 'SG-SF'], start=1, end=2), legend='Position')

    plot.yaxis.axis_label = "Log Salary"
    plot.xaxis.axis_label = X_label
    plot.legend.location = "bottom_right"
    plot.title.align = "center"
    plot.background_fill_color= '#f2f2f2'
    show(plot, notebook_handle=True)
    push_notebook()


interact(update, X_Variable = ["Age", "PSG"], Age_Jitter =(0.0, 0.7))

ValueError: PATCH-DOC message requires at least one event

<function __main__.update>

# Question 3: Clustering (15 pts)
## Part a

This problem explores scaling, clustering, and one method of evaluating the quality of a clustering. You will also examine some of the issues in k-means clustering, and some ways to mitigate these problems. You will be using Fisher's Iris dataset available in sklearn.

We wish to cluster the dataset to find any similar groups of flowers. However, we are not certain that the scales of the various features are well-suited for clustering. We will use scipy's k-means clustering package for this problem.

When calculating the cluster purity, use the following equations:

$ClassPurity(C_i$) = $\frac{1}{|C_i|}\max_j(|C_i|_{class=j})$

$NetPurity$ = $\sum_{i=1}^k\frac{|C_i|}{|D|}purity(C_i)$

where $|C_i|$ is the total number of data points assigned to cluster $C_i$, $|C_i|_{class=j}$ is the number of data points from class $j$ assigned to cluster $C_i$ and $D$ refers to the whole dataset.

1. Cluster the data into 4 clusters, using K-means and calculate the NetPurity for your solution.
2. Now linearly scale each feature so that the values range from 0 to 1. Cluster the data using the k-means algorithm (as before), and calculate the cluster purity for the clustering. Report the calculations you used to scale the features, as well as the NetPurity.
3. Linearly scale the original dataset features so that the distribution of values for each feature has a mean of 0 and a standard deviation of 1. Cluster the data as before, and report the NetPurity obtained.



In [3]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn import datasets

## Part b

Run both single link and complete link hierarchical clustering algorithms on the iris dataset, and submit the two dendograms. 

1. For each dendogram, visually inspect it to suggest what value(s) of $k$ (between 1 and 6) seem reasonable to choose for this dataset. 
2. Do you observe any difference in the structure of the 2 dendograms? How do you explain this difference?

In [4]:
% matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn import datasets

## Part c

1. What is the cost function minimised in the k-means clustering algorithm?
2. Explain how this can be generalized to be of the form $J = \sum_{i=1}^k n_i s_i$, where $s_i$ is the average squared (Euclidean) distance between pairs of points in cluster $i$ (every point is compared with every point - including itself - in the same cluster, so we have $n_i^2$ comparisions that are averaged over to get $s_i$), 
and $n_i$ is the total number of points in cluster $i$.
3. What would the benefit of such a generalized expression be?

1. Standard Expression, $J = \sum_{i=1}^k \sum_{x \in S_i} ||x-\mu_i||^2$, where \mu_i is the mean of the points in S_i
2. Generalized expression, 
    $J = \sum_{i=1}^k n_i * (||x-\mu_i||^2 /n_i)$ where n_i is the number of samples in each cluster
    
    $J = \sum_{i=1}^k n_i * s_i $ where s_i is the variance within each cluster

3. Benefit: $s_i$ can now be replaced with any similarity metric based on the clustering problem requirements